In [3]:
!pip install emoji
!pip install langdetect


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ------------------------------------- 981.5/981.5 kB 23.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993362 sha256=3f8bc2760e5a1bb5484b34bca7c1ee3942e13bceee96e8aa240378c0906d8f25
  Stored in directory: c:\users\ksenialuschevaext\appdata\local\pip\cache\wheels\c1\67\88\e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import json
import os
import emoji
import re
import pandas as pd
from tqdm import tqdm
from langdetect import detect

In [18]:
def clean_text(text):
    """Apply common NLP preprocessing to text"""
    if not text or not isinstance(text, str):
        return ""
    
    # Lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove punctuation (keeping apostrophes for contractions)
    text = re.sub(r'[^\w\s\']', '', text)
    
    # Remove extra whitespace and newlines
    text = ' '.join(text.split())
    
    return text.strip()

In [19]:
def process_json_file(json_path, min_words=100):
    """Process a single JSON file and return cleaned text if valid"""
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        text = data.get('text', '')
        cleaned = clean_text(text)
        
        # Check word count
        word_count = len(cleaned.split())
        if word_count >= min_words:
            return cleaned
        return None
    
    except Exception as e:
        print(f"Error processing {json_path}: {str(e)}")
        return None

In [21]:
def is_russian(text):
    try:
        return detect(text) == 'ru'
    except:
        return False

In [22]:
def process_directory(input_dir, output_csv):
    """Process all files in directory and save to CSV"""
    records = []
    
    for root, _, files in os.walk(input_dir):
        for file in tqdm(files, desc="Processing files"):
            file_path = os.path.join(root, file)
            print(f"file path is: {file_path}")
            if file.endswith('.json'):
                print(f"json found: {file}")
                text = process_json_file(file_path)
            else:
                continue

            if is_russian(text) == False:
                print(f"Skipping text {file}! It is not in russian language")
                continue
            
            if text:
                records.append({'text': text, 'source': file})
    
    # Create DataFrame and save to CSV
    if records:
        df = pd.DataFrame(records)
        df.to_csv(output_csv, index=False, encoding='utf-8')
        print(f"Saved {len(df)} records to {output_csv}")
    else:
        print("No valid records found")


In [ ]:
process_directory('/Users/Klushcheva/Documents/thesis/proza', 'processed_proza_texts.csv')